In [1]:
library(RPostgreSQL)
library(twang)
library(Matching)
library(tidyverse)

Loading required package: DBI
Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1

In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
full_data <- dbGetQuery(con, "select * from merged_data")

In [5]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [6]:
to_factor <- function(x) {
    res <- (x %>% as.factor %>% as.integer) - 1
    if(length(na.omit(unique(res))) <= 1) return(factor(res, levels = c(0, 1)))
    return(factor(res))
}

In [7]:
factor_vars <- full_data %>%
    names %>%
    grep("flag|abnormal|icd|sedative", ., value = TRUE) %>%
    c("gender", "first_careunit", "echo", "vent", "vaso",
      "icu_adm_weekday", "icu_adm_hour", "mort_28_day")
factor_vars

[1] "icd_chf"                        "icd_afib"                      
 [3] "icd_renal"                      "icd_liver"                     
 [5] "icd_copd"                       "icd_cad"                       
 [7] "icd_stroke"                     "icd_malignancy"                
 [9] "vs_temp_flag"                   "vs_map_flag"                   
[11] "vs_cvp_flag"                    "vs_heart_rate_flag"            
[13] "lab_hemoglobin_flag"            "lab_hemoglobin_abnormal"       
[15] "lab_wbc_flag"                   "lab_wbc_abnormal"              
[17] "lab_lactate_flag"               "lab_lactate_abnormal"          
[19] "lab_po2_flag"                   "lab_po2_abnormal"              
[21] "lab_troponin_flag"              "lab_troponin_abnormal"         
[23] "lab_potassium_flag"             "lab_potassium_abnormal"        
[25] "lab_creatinine_kinase_flag"     "lab_creatinine_kinase_abnormal"
[27] "lab_creatinine_flag"            "lab_creatinine_abnormal"       
[29] "lab_pco2_flag"                  "lab_pco2_abnormal"             
[31] "lab_bnp_flag"                   "lab_bnp_abnormal"              
[33] "lab_bicarbonate_flag"           "lab_bicarbonate_abnormal"      
[35] "lab_bun_flag"                   "lab_bun_abnormal"              
[37] "lab_platelet_flag"              "lab_platelet_abnormal"         
[39] "lab_sodium_flag"                "lab_sodium_abnormal"           
[41] "lab_chloride_flag"              "lab_chloride_abnormal"         
[43] "lab_ph_flag"                    "lab_ph_abnormal"               
[45] "sedative"                       "gender"                        
[47] "first_careunit"                 "echo"                          
[49] "vent"                           "vaso"                          
[51] "icu_adm_weekday"                "icu_adm_hour"                  
[53] "mort_28_day"

In [8]:
full_data <- full_data %>%
    mutate(echo_int = as.integer(echo)) %>%
    mutate(mort_28_day_int = as.integer(mort_28_day)) %>%
    mutate_at(factor_vars, to_factor)
full_data %>% pull(echo) %>% head

[1] 0 0 0 1 0 0
Levels: 0 1

In [9]:
feature_names <- full_data %>%
    names %>%
    keep(grepl("vs|lab|icd|age|gender|weight|saps|sofa|elix_score|vent|vaso|icu_adm|sedative", .)) %>%
    discard(grepl("vs|lab", .) & grepl("flag", .) & !grepl("bnp|troponin|kinase", .)) %>%
    discard(grepl("bnp|troponin|kinase", .) & !grepl("flag", .)) %>%
    discard(grepl("min|max", .)) %>%
    discard(grepl("abnormal", .))
feature_names
length(feature_names)

[1] "age"                        "gender"                    
 [3] "weight"                     "saps"                      
 [5] "sofa"                       "elix_score"                
 [7] "vent"                       "vaso"                      
 [9] "icu_adm_weekday"            "icu_adm_hour"              
[11] "icd_chf"                    "icd_afib"                  
[13] "icd_renal"                  "icd_liver"                 
[15] "icd_copd"                   "icd_cad"                   
[17] "icd_stroke"                 "icd_malignancy"            
[19] "vs_temp_first"              "vs_map_first"              
[21] "vs_cvp_first"               "vs_heart_rate_first"       
[23] "lab_hemoglobin_first"       "lab_wbc_first"             
[25] "lab_lactate_first"          "lab_po2_first"             
[27] "lab_troponin_flag"          "lab_potassium_first"       
[29] "lab_creatinine_kinase_flag" "lab_creatinine_first"      
[31] "lab_pco2_first"             "lab_bnp_flag"              
[33] "lab_bicarbonate_first"      "lab_bun_first"             
[35] "lab_platelet_first"         "lab_sodium_first"          
[37] "lab_chloride_first"         "lab_ph_first"              
[39] "sedative"

[1] 39

In [10]:
features <- full_data %>%
    select(!!!rlang::syms(feature_names))
head(features)

age,gender,weight,saps,sofa,elix_score,vent,vaso,icu_adm_weekday,icu_adm_hour,⋯,lab_creatinine_first,lab_pco2_first,lab_bnp_flag,lab_bicarbonate_first,lab_bun_first,lab_platelet_first,lab_sodium_first,lab_chloride_first,lab_ph_first,sedative
62.67646,1,74.3,25,5,5,1,0,2,1,⋯,3.7,32,0,22,208,313,160,123,7.45,1
86.76186,0,NA,13,1,10,0,0,4,1,⋯,0.9,NA,0,27,17,189,139,105,NA,0
56.08904,1,65.0,18,5,14,1,0,6,1,⋯,0.9,32,0,24,15,231,144,108,7.49,0
45.91093,1,NA,16,9,13,0,0,3,14,⋯,0.8,30,0,20,19,28,134,100,7.44,0
59.38693,1,91.4,13,3,22,0,0,3,14,⋯,0.7,NA,0,26,7,40,138,103,NA,0
300.00345,0,55.0,25,5,0,0,0,6,20,⋯,0.8,NA,0,20,19,249,147,118,NA,0


In [11]:
label_name <- "echo"

In [12]:
label <- full_data %>% pull(echo)
head(label)

[1] 0 0 0 1 0 0
Levels: 0 1

In [13]:
fml <- feature_names %>%
    c("echo", .) %>%
    paste(collapse = " + ") %>%
    sprintf("mort_28_day ~ %s", .)
fml

[1] "mort_28_day ~ echo + age + gender + weight + saps + sofa + elix_score + vent + vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_temp_first + vs_map_first + vs_cvp_first + vs_heart_rate_first + lab_hemoglobin_first + lab_wbc_first + lab_lactate_first + lab_po2_first + lab_troponin_flag + lab_potassium_first + lab_creatinine_kinase_flag + lab_creatinine_first + lab_pco2_first + lab_bnp_flag + lab_bicarbonate_first + lab_bun_first + lab_platelet_first + lab_sodium_first + lab_chloride_first + lab_ph_first + sedative"

In [14]:
unweighted <- glm(as.formula(fml), data = full_data, family = binomial, na.action = na.exclude)
summary(unweighted)
exp(cbind(OR = coef(unweighted), confint(unweighted)))


Call:
glm(formula = as.formula(fml), family = binomial, data = full_data, 
    na.action = na.exclude)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1335  -0.7365  -0.4171   0.7583   2.8996  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  5.4443648  8.7492730   0.622 0.533768    
echo1                       -0.3311306  0.1483746  -2.232 0.025633 *  
age                          0.0041990  0.0012056   3.483 0.000496 ***
gender1                      0.2742175  0.1435749   1.910 0.056143 .  
weight                      -0.0088982  0.0031475  -2.827 0.004698 ** 
saps                         0.0800412  0.0172166   4.649 3.33e-06 ***
sofa                         0.1605250  0.0259602   6.184 6.27e-10 ***
elix_score                   0.0295440  0.0114913   2.571 0.010141 *  
vent1                        0.2625683  0.2672348   0.983 0.325835    
vaso1                        0.0260530  0.1752765   0.149

Waiting for profiling to be done...


,OR,2.5 %,97.5 %
(Intercept),231.4502180,7.333704e-06,5.945866e+09
echo1,0.7181113,5.366233e-01,9.603771e-01
age,1.0042078,1.001840e+00,1.006595e+00
gender1,1.3155009,9.932485e-01,1.744381e+00
weight,0.9911413,9.849634e-01,9.972043e-01
saps,1.0833317,1.047576e+00,1.120777e+00
sofa,1.1741271,1.116359e+00,1.236045e+00
elix_score,1.0299848,1.007020e+00,1.053466e+00
vent1,1.3002652,7.700479e-01,2.197377e+00
vaso1,1.0263953,7.283564e-01,1.448777e+00


In [15]:
fml <- feature_names %>%
    paste(collapse = " + ") %>%
    sprintf("echo_int ~ %s", .)
fml

[1] "echo_int ~ age + gender + weight + saps + sofa + elix_score + vent + vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_temp_first + vs_map_first + vs_cvp_first + vs_heart_rate_first + lab_hemoglobin_first + lab_wbc_first + lab_lactate_first + lab_po2_first + lab_troponin_flag + lab_potassium_first + lab_creatinine_kinase_flag + lab_creatinine_first + lab_pco2_first + lab_bnp_flag + lab_bicarbonate_first + lab_bun_first + lab_platelet_first + lab_sodium_first + lab_chloride_first + lab_ph_first + sedative"

In [16]:
echo_ps_ate <- ps(as.formula(fml),
                  data = full_data,
                  interaction.depth = 2,
                  shrinkage = 0.01,
                  perm.test.iters = 0,
                  estimand = "ATE",
                  verbose = FALSE,
                  stop.method = c("es.mean", "es.max", "ks.mean", "ks.max"),
                  n.trees = 10000,
                  train.fraction = 0.8,
                  cv.folds = 3,
                  n.cores = 8)

In [17]:
pred <- echo_ps_ate$ps$es.mean.ATE
full_data <- full_data %>% mutate(ps = pred)
ROCR::performance(ROCR::prediction(pred, label), "auc")@y.values %>% first

[1] 0.8186991

In [18]:
ft_importance <- summary(echo_ps_ate$gbm.obj,
                         n.trees = echo_ps_ate$desc$es.mean.ATE$n.trees,
                         plot = FALSE)

In [19]:
full_data <- full_data %>%
    mutate(ps_weight = get.weights(echo_ps_ate, stop.method = "es.mean"))

In [20]:
saveRDS(full_data, file = file.path(data_dir, "full_data_ps.rds"))

In [21]:
saveRDS(ft_importance, file = file.path(data_dir, "feature_importance.rds"))

In [22]:
primary_ipw <- glm(mort_28_day ~ echo, data = full_data,
                   weights = full_data$ps_weight, family = binomial)
summary(primary_ipw)
exp(cbind(OR = coef(primary_ipw), confint(primary_ipw)))

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”


Call:
glm(formula = mort_28_day ~ echo, family = binomial, data = full_data, 
    weights = full_data$ps_weight)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2760  -1.0703  -0.9453   1.7287   4.5574  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.89603    0.02950 -30.375  < 2e-16 ***
echo1       -0.18066    0.04251  -4.249 2.14e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 13120  on 6161  degrees of freedom
Residual deviance: 13102  on 6160  degrees of freedom
AIC: 12757

Number of Fisher Scoring iterations: 4


Waiting for profiling to be done...
Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”Warning

,OR,2.5 %,97.5 %
(Intercept),0.4081856,0.3851623,0.4323822
echo1,0.8347150,0.7679424,0.9072201


In [23]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)
# design_echo_ps_ate <- svydesign(ids = ~ icustay_id, data = full_data)

In [24]:
fml <- feature_names %>%
    c(label_name, .) %>%
    paste(collapse = " + ") %>%
    sprintf("mort_28_day ~ %s", .)
fml

[1] "mort_28_day ~ echo + age + gender + weight + saps + sofa + elix_score + vent + vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_temp_first + vs_map_first + vs_cvp_first + vs_heart_rate_first + lab_hemoglobin_first + lab_wbc_first + lab_lactate_first + lab_po2_first + lab_troponin_flag + lab_potassium_first + lab_creatinine_kinase_flag + lab_creatinine_first + lab_pco2_first + lab_bnp_flag + lab_bicarbonate_first + lab_bun_first + lab_platelet_first + lab_sodium_first + lab_chloride_first + lab_ph_first + sedative"

In [25]:
logi <- svyglm(as.formula(fml),
               family = quasibinomial,
               design = design_echo_ps_ate)

In [26]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 10.2998082  9.0375906   1.140 0.254613    
echo1                       -0.3482795  0.1503699  -2.316 0.020687 *  
age                          0.0045672  0.0013126   3.480 0.000517 ***
gender1                      0.2110474  0.1561738   1.351 0.176788    
weight                      -0.0106926  0.0034538  -3.096 0.001999 ** 
saps                         0.0711413  0.0182289   3.903 9.95e-05 ***
sofa                         0.1825126  0.0265935   6.863 9.92e-12 ***
elix_score                   0.0254845  0.0122369   2.083 0.037461 *  
vent1                        0.4686153  0.2873921   1.631 0.103194    
vaso1                       -0.0193613  0.1850163  -0.105 0.916670    
icu_adm_weekday1         

In [27]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),2.972692e+04,0.0006030298,1.465416e+12
echo1,7.059016e-01,0.5257134236,9.478491e-01
age,1.004578e+00,1.0019965629,1.007165e+00
gender1,1.234971e+00,0.9093296230,1.677228e+00
weight,9.893644e-01,0.9826897595,9.960844e-01
saps,1.073733e+00,1.0360478734,1.112789e+00
sofa,1.200229e+00,1.1392730225,1.264447e+00
elix_score,1.025812e+00,1.0015017822,1.050712e+00
vent1,1.597780e+00,0.9096796103,2.806374e+00
vaso1,9.808249e-01,0.6825041867,1.409541e+00


In [28]:
ps_matches <- Match(Y = NULL, Tr = full_data$echo_int, X = full_data$ps, M = 1,
                    estimand = "ATT", caliper = 0.1,
                    exact = FALSE, replace = FALSE)

In [29]:
summary(ps_matches)


Estimate...  0 
SE.........  0 
T-stat.....  NaN 
p.val......  NA 

Original number of observations..............  6162 
Original number of treated obs...............  3063 
Matched number of observations...............  1585 
Matched number of observations  (unweighted).  1585 

Number of obs dropped by 'exact' or 'caliper'  1478 



In [30]:
tab <- table(full_data$mort_28_day[ps_matches$index.treated],
             full_data$mort_28_day[ps_matches$index.control],
             dnn = c("Echo", "Control"))
tab

    Control
Echo   0   1
   0 839 353
   1 275 118

In [31]:
fisher.test(tab)


	Fisher's Exact Test for Count Data

data:  tab
p-value = 0.8988
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 0.7873963 1.3162819
sample estimates:
odds ratio 
   1.01985 


In [32]:
mcnemar.test(tab)


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 9.4411, df = 1, p-value = 0.002122
